## Example how to combine shielding with rllibs ppo algorithm.

In [4]:
import gymnasium as gym

import minigrid

from ray.tune import register_env
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.logger import pretty_print
from ray.rllib.models import ModelCatalog


from torch_action_mask_model import TorchActionMaskModel
from wrappers import OneHotShieldingWrapper, MiniGridShieldingWrapper
from shieldhandlers import MiniGridShieldHandler, create_shield_query
  

In [5]:
def shielding_env_creater(config):
    name = config.get("name", "MiniGrid-LavaCrossingS9N1-v0")
    framestack = config.get("framestack", 4)
    
    shield_creator = MiniGridShieldHandler("grid.txt", "./main", "grid.prism", "Pmax=? [G !\"AgentIsInLavaAndNotDone\"]")
    
    env = gym.make(name)
    env = MiniGridShieldingWrapper(env, shield_creator=shield_creator, shield_query_creator=create_shield_query ,mask_actions=True)
    env = OneHotShieldingWrapper(env, config.vector_index if hasattr(config, "vector_index") else 0,
                                 framestack=framestack)
    
    return env


def register_minigrid_shielding_env():
    env_name = "mini-grid-shielding"
    register_env(env_name, shielding_env_creater)
    ModelCatalog.register_custom_model(
        "shielding_model", 
        TorchActionMaskModel)

In [ ]:
register_minigrid_shielding_env()


config = (PPOConfig()
    .rollouts(num_rollout_workers=1)
    .resources(num_gpus=0)
    .environment(env="mini-grid-shielding", env_config={"name": "MiniGrid-LavaCrossingS9N1-v0"})
    .framework("torch")
    .rl_module(_enable_rl_module_api = False)
    .training(_enable_learner_api=False ,model={
        "custom_model": "shielding_model"
    }))


algo = (
    config.build()
)
        
for i in range(30):
    result = algo.train()
    print(pretty_print(result))

    if i % 5 == 0:
        print("Saving checkpoint")
        checkpoint_dir = algo.save()
        print(f"Checkpoint saved in directory {checkpoint_dir}")